In [4]:
# ================================================================
# SymPy implementation of central-idempotent projectors
# for the k-subset representation of S_n (two-row partitions only)
# ================================================================
import itertools as it
import math
from collections import Counter
from sympy import Matrix, Rational, eye, sqrt

# -----------------------------------
# Exact Gram–Schmidt implementation
# -----------------------------------
def gram_schmidt(vectors, orthonormal=False):
    """
    Simple exact Gram–Schmidt process for SymPy column vectors.
    vectors: list of sympy.Matrix (each column vector)
    orthonormal: if True, normalizes each vector to unit length
    """
    ortho = []
    for v in vectors:
        w = v
        for u in ortho:
            w = w - (u.dot(v) / u.dot(u)) * u
        if w.norm() == 0:
            continue
        if orthonormal:
            w = w / sqrt(w.dot(w))
        ortho.append(w)
    return ortho

# -----------------------------------
# Combinatorial helpers
# -----------------------------------

def dim_two_row(n, i):
    """Dimension of two-row Specht module (n-i,i)."""
    return math.comb(n, i) - (math.comb(n, i-1) if i > 0 else 0)

def all_perms(n):
    """All permutations of {0,...,n-1} as tuples."""
    return list(it.permutations(range(n)))

def cycle_type(p):
    """Cycle type of permutation p as a partition tuple."""
    n = len(p)
    seen = [False]*n
    cyc = []
    for i in range(n):
        if not seen[i]:
            j=i; L=0
            while not seen[j]:
                seen[j] = True
                j = p[j]
                L += 1
            cyc.append(L)
    cyc.sort(reverse=True)
    return tuple(cyc)

def conjugacy_classes_with_members(n):
    """Return [{type, size, members}] for S_n."""
    classes = {}
    for g in all_perms(n):
        mu = cycle_type(g)
        classes.setdefault(mu, []).append(g)
    out = []
    for mu, members in classes.items():
        out.append({"type": mu, "size": len(members), "members": members})
    return out

# -----------------------------------
# k-subset permutation representation
# -----------------------------------

def k_subsets(n, k):
    return [tuple(c) for c in it.combinations(range(n), k)]

def rho_matrix(n, k, sigma):
    """Permutation matrix of sigma acting on k-subsets."""
    Xk = k_subsets(n, k)
    idx = {A:i for i,A in enumerate(Xk)}
    m = len(Xk)
    M = [[0]*m for _ in range(m)]
    for j,A in enumerate(Xk):
        B = tuple(sorted(sigma[a] for a in A))
        i = idx[B]
        M[i][j] = 1
    return Matrix(M)

# -----------------------------------
# Two-row character polynomials
# -----------------------------------

def chi_two_row(n, i, cycle_type_part):
    """Character χ^{(n-i,i)}(σ) for σ of given cycle type."""
    c = Counter(cycle_type_part)
    X1, X2, X3 = c[1], c[2], c[3]
    if i == 0:  # trivial
        return Rational(1)
    elif i == 1:
        return Rational(X1 - 1)
    elif i == 2:
        return Rational((X1*(X1-1))/2 + X2 - X1)
    elif i == 3:
        return Rational((X1*(X1-1)*(X1-2))/6 + X2*(X1-1) + X3 - (X1*(X1-1))/2 - X2)
    else:
        raise NotImplementedError("Extend for larger i if needed.")

# -----------------------------------
# Central idempotent projectors (exact)
# -----------------------------------

def projectors_two_row_vnk(n, k):
    """
    Build exact projectors p_{(n-i,i)} on R[X_k].
    Returns {i: projector Matrix}, {i: orthonormal basis Matrix}.
    """
    basis = k_subsets(n, k)
    m = len(basis)
    classes = conjugacy_classes_with_members(n)

    # Precompute ρ(g)
    rho = {}
    for C in classes:
        for g in C["members"]:
            if g not in rho:
                rho[g] = rho_matrix(n, k, g)

    proj, Qblocks = {}, {}
    for i in range(0, k+1):
        dimL = dim_two_row(n, i)
        M = Matrix.zeros(m)

        for C in classes:
            mu = C["type"]
            chi = chi_two_row(n, i, mu)
            # true class sum
            S_C = sum((rho[g] for g in C["members"]), Matrix.zeros(m))
            M += chi * S_C

        p = (Rational(dimL, math.factorial(n))) * M
        p = (p + p.T) / 2   # ensure symmetry
        proj[i] = p

        # Eigen decomposition for eigenvalue 1 subspace
        evects = p.eigenvects()
        eig1 = [v for (val, mult, vecs) in evects if val == 1 for v in vecs]
        if len(eig1) < dimL:
            eig1 += [v for (val, mult, vecs) in evects if abs(float(val)-1)<1e-8 for v in vecs]
        cols = [Matrix(v) for v in eig1[:dimL]]
        ortho_cols = gram_schmidt(cols, orthonormal=True)
        Qblocks[i] = Matrix.hstack(*ortho_cols)

    return proj, Qblocks, basis

# -----------------------------------
# Verification & block diagonalization
# -----------------------------------

def assemble_Q(Qblocks):
    return Matrix.hstack(*[Qblocks[i] for i in sorted(Qblocks.keys())])

def verify_projectors(proj, dims):
    ok = True
    keys = sorted(proj.keys())
    m = proj[keys[0]].rows
    I = eye(m)

    # Check idempotence, orthogonality, completeness
    for i in keys:
        if not (proj[i]*proj[i]).equals(proj[i]):
            print(f"[warn] Projector {i} not idempotent."); ok=False
        tr = proj[i].trace()
        if tr != dims[i]:
            print(f"[warn] trace(p_{i})={tr} differs from dim {dims[i]}."); ok=False
    S = sum((proj[i] for i in keys), Matrix.zeros(m))
    if not S.equals(I):
        print("[warn] Sum of projectors not identity."); ok=False
    for a in keys:
        for b in keys:
            if a<b and not (proj[a]*proj[b]).is_zero_matrix:
                print(f"[warn] Projectors {a},{b} not orthogonal."); ok=False
    return ok

def block_diagonalize(n, k, Q, sigma):
    R = rho_matrix(n, k, sigma)
    return (Q.T * R * Q).applyfunc(lambda x: x.simplify())

# -----------------------------------
# Example run
# -----------------------------------

if __name__ == "__main__":
    n, k = 4, 2   # small example (S4 on 2-subsets)
    proj, Qblocks, basis = projectors_two_row_vnk(n, k)
    dims = {i: dim_two_row(n, i) for i in range(0, k+1)}
    print("Block dimensions:", dims)

    ok = verify_projectors(proj, dims)
    print("Verification passed:", ok)

    Q = assemble_Q(Qblocks)
    print("Q.T*Q =", (Q.T*Q).simplify())

    # Check block diagonalization for σ=(0 1)
    sigma = list(range(n)); sigma[0], sigma[1] = sigma[1], sigma[0]; sigma = tuple(sigma)
    N = block_diagonalize(n, k, Q, sigma)
    print("Block-diagonal form of ρ((0 1)):")
    print(N)


Block dimensions: {0: 1, 1: 3, 2: 2}
Verification passed: True
Q.T*Q = None
Block-diagonal form of ρ((0 1)):
Matrix([[1, 0, 0, 0, 0, 0], [0, 0, -1, 0, 0, 0], [0, -1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, -1, 0], [0, 0, 0, 0, 0, 1]])


Matrix([
[   0,  sqrt(3)/3],
[ 1/2, -sqrt(3)/6],
[-1/2, -sqrt(3)/6],
[-1/2, -sqrt(3)/6],
[ 1/2, -sqrt(3)/6],
[   0,  sqrt(3)/3]])